In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', 500)

In [2]:
mapp   = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t', index_col='ensembl')
n2e    = pd.read_csv('../tissue_expression/nowasaki2ensembl.tsv', sep='\t')
pheno  = pd.read_csv('../data/Nowakowski/meta.tsv', sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tpm    = pd.read_csv('../data/Nowakowski/tpm.tsv', sep='\t')
tpm_backup = tpm.copy()
# tpm = tpm_backup.copy()
tpm = tpm.set_index('ensembl')


In [4]:
n2e = n2e.set_index('nowasaki', drop=False)
pheno.loc[pheno.WGCNAcluster.apply(lambda s: s!=s), 'WGCNAcluster'] = 'None'
pheno = pheno.set_index('Cell', drop=False)

In [5]:
tpm.columns = pheno.loc[tpm.columns, 'WGCNAcluster']

In [6]:
# create tpm by cell type
tpm_by_cell_type = pd.DataFrame()

for s in list(set(tpm.columns)):
    tpm_by_cell_type[s] = tpm[s].T.median()

In [7]:
summ = pd.DataFrame(index=tpm_by_cell_type.index)
summ['median'] = 0
summ['mean'] = 0
summ['std'] = 0

for gene in summ.index:
    summ.loc[gene, 'median'] = tpm_by_cell_type.loc[gene].median()
    summ.loc[gene, 'mean'] = tpm_by_cell_type.loc[gene].mean()
    summ.loc[gene, 'std'] = tpm_by_cell_type.loc[gene].std()

In [8]:
def is_enriched_in_tissue(tpm, std_gene, med_gene):
    if tpm > med_gene + 2* std_gene:
        return True
    return False


summ['enriched_tissues'] = '.'

for gene in summ.index:
    std_gene = summ.loc[gene, 'std']
    med_gene = summ.loc[gene, 'median']
    enrichements =  tpm_by_cell_type.loc[gene].apply(is_enriched_in_tissue, std_gene=std_gene, med_gene=med_gene)
    enrichements = enrichements[enrichements].index.tolist()

    enrichements = ';'.join(enrichements)
    summ.at[gene, 'enriched_tissues'] = enrichements

In [10]:
summ.to_csv('cell_type_preferential_tissue_expression.tsv', sep='\t')

In [20]:
df2 = df.copy()
for col in df2.columns:
    df2[col] = 0

In [21]:
for gene in summ.index[:]:
    tissues = summ.at[gene, 'enriched_tissues']
    tissues = tissues.split(',')
    for tissue in tissues:
        df2.at[gene, tissue] = 1


KeyboardInterrupt: 

In [ ]:
df2.index.Name = 'ensembl'

In [ ]:
df2 = df2.drop('', axis=1)

In [ ]:
df2

In [ ]:
df2.to_csv('cell_type_preferential_expression.tsv', sep='\t')